In [6]:
#Stock price prediction with sentiment analysis 
#Data Scraping
import requests
from bs4 import BeautifulSoup
!pip uninstall numpy pandas -y
!pip cache purge

!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall pandas

# Optional: Display pandas dataframes nicely in Jupyter
import pandas as pd
pd.set_option('display.max_colwidth', None)


Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
Files removed: 38
Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/numpy/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/numpy/ (Caused by SSLError(SSLCertVerificationError("The certificate's CN name does not match the passed value."))) - skipping


ERROR: Could not find a version that satisfies the requirement numpy (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for numpy


Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/pandas/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pandas/ (Caused by SSLError(SSLCertVerificationError("The certificate's CN name does not match the passed value."))) - skipping


ERROR: Could not find a version that satisfies the requirement pandas (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pandas


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#setting up selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Setup ChromeDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Runs Chrome in the background (no GUI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Launch WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

def convert_relative_date(date_str):
    """Convert relative or absolute dates into a datetime object, fixing 'Sept' parsing."""
    now = datetime.now()
    lower = date_str.lower().strip()
    
    if "ago" in lower:
        import re
        match = re.search(r'(\d+)\s*(minute|minutes|hour|hours|day|days)', lower)
        if match:
            quantity = int(match.group(1))
            unit = match.group(2)
            if 'minute' in unit:
                delta = timedelta(minutes=quantity)
            elif 'hour' in unit:
                delta = timedelta(hours=quantity)
            elif 'day' in unit:
                delta = timedelta(days=quantity)
            else:
                delta = timedelta(0)
            return now - delta
        return now
    elif lower == "just now":
        return now
    elif lower == "yesterday":
        return now - timedelta(days=1)
    else:
        month_map = {"Sept": "Sep"}  # Fix for 'Sept' issue
        for wrong, correct in month_map.items():
            date_str = date_str.replace(wrong, correct)

        for fmt in ("%b %d, %Y", "%d %b %Y"):
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        print("Error parsing date string:", date_str)
        return now

def scrape_date_range(query, start_date, end_date, driver):
    """Scrapes Google News articles within a date range."""
    cd_min = start_date.strftime("%m/%d/%Y")
    cd_max = end_date.strftime("%m/%d/%Y")
    query_encoded = query.replace(" ", "+")
    url = f"https://www.google.com/search?q={query_encoded}&tbs=cdr:1,cd_min:{cd_min},cd_max:{cd_max}&tbm=nws"
    
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.SoaBEf")))
    except:
        print(f"No articles found for {cd_min} to {cd_max}")
        return []
    
    for _ in range(3):  # Scroll to load more
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    articles = driver.find_elements(By.CSS_SELECTOR, "div.SoaBEf")
    data = []
    
    for article in articles:
        try:
            title = article.find_element(By.CSS_SELECTOR, "div.MBeuO").text
            link = article.find_element(By.TAG_NAME, "a").get_attribute("href")
            date_str = article.find_element(By.CSS_SELECTOR, "div.OSrXXb span").text
            pub_date = convert_relative_date(date_str)
            data.append({"Title": title, "Link": link, "Date": pub_date})
        except Exception as e:
            print("Error processing an article:", e)
    
    return data

# Set up WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

service = Service("chromedriver.exe") 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# Scraping process
overall_start = datetime(2015, 2, 1)
overall_end = datetime(2025, 3, 1)
delta = timedelta(days=7)
query = "stock market news"
all_data = []

current_start = overall_start
while current_start < overall_end:
    current_end = current_start + delta
    if current_end > overall_end:
        current_end = overall_end
    print(f"Scraping {current_start.date()} to {current_end.date()}")
    data_chunk = scrape_date_range(query, current_start, current_end, driver)
    all_data.extend(data_chunk)
    current_start = current_end + timedelta(days=1)
    time.sleep(2)

driver.quit()


Scraping 2015-02-01 to 2015-02-08
Scraping 2015-02-09 to 2015-02-16
Scraping 2015-02-17 to 2015-02-24
Scraping 2015-02-25 to 2015-03-04
Scraping 2015-03-05 to 2015-03-12
Scraping 2015-03-13 to 2015-03-20
Scraping 2015-03-21 to 2015-03-28
Scraping 2015-03-29 to 2015-04-05
Scraping 2015-04-06 to 2015-04-13
Scraping 2015-04-14 to 2015-04-21
Scraping 2015-04-22 to 2015-04-29
Scraping 2015-04-30 to 2015-05-07
Scraping 2015-05-08 to 2015-05-15
Scraping 2015-05-16 to 2015-05-23
Scraping 2015-05-24 to 2015-05-31
Scraping 2015-06-01 to 2015-06-08
Scraping 2015-06-09 to 2015-06-16
Scraping 2015-06-17 to 2015-06-24
Scraping 2015-06-25 to 2015-07-02
Scraping 2015-07-03 to 2015-07-10
Scraping 2015-07-11 to 2015-07-18
Scraping 2015-07-19 to 2015-07-26
Scraping 2015-07-27 to 2015-08-03
Scraping 2015-08-04 to 2015-08-11
Scraping 2015-08-12 to 2015-08-19
Scraping 2015-08-20 to 2015-08-27
Scraping 2015-08-28 to 2015-09-04
Scraping 2015-09-05 to 2015-09-12
Scraping 2015-09-13 to 2015-09-20
Scraping 2015-

In [22]:
print(f"Total articles scraped: {len(all_data)}")

Total articles scraped: 3015


In [12]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait                #not processed
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Selenium WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Process in 2-year batches
start_year = 2015
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2015 - 2016 (877 articles)

[2025-03-02 20:34:04] Fetching: https://www.business-standard.com/article/markets/nse-suspends-trading-in-companies-of-two-defunct-bourses-115020400938_1.html
✅ Completed in 3.25 seconds

[2025-03-02 20:34:07] Fetching: https://m.economictimes.com/markets/stocks/news/chandrakant-sampat-a-life-spent-spotting-value-in-indian-markets/articleshow/46102338.cms
✅ Completed in 81.49 seconds

[2025-03-02 20:35:28] Fetching: https://www.moneycontrol.com/news/business/markets/chandrakant-sampat-pioneervalue-investing-dies-at-86-1287003.html
✅ Completed in 68493.16 seconds

[2025-03-03 15:37:01] Fetching: https://www.business-standard.com/article/markets/trading-firms-on-closed-madras-and-madhya-pradesh-exchanges-out-of-nse-too-115020400165_1.html
✅ Completed in 4.07 seconds

[2025-03-03 15:37:05] Fetching: https://www.bbc.com/news/business-31144009
✅ Completed in 13.93 seconds

[2025-03-03 15:37:19] Fetching: https://www.livemint.com/Politics/NXRU

KeyboardInterrupt: 

In [13]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Selenium WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2018
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2018 - 2019 (915 articles)

[2025-03-03 20:08:15] Fetching: https://scroll.in/article/867738/the-morning-panic-in-indian-stocks-was-not-a-one-off-and-more-is-yet-to-come
✅ Completed in 46.59 seconds

[2025-03-03 20:09:01] Fetching: https://www.nbcnews.com/business/markets/new-fed-chair-sworn-dow-drop-450-points-n844826
✅ Completed in 50.05 seconds

[2025-03-03 20:09:51] Fetching: https://www.financialexpress.com/market/live-budget-2018-market-sensex-nifty-bse-corporate-sector-nse-indian-stock-market-live-on-budget-day-fm-arun-jaitley-1039188/
✅ Completed in 4.79 seconds

[2025-03-03 20:09:56] Fetching: https://www.nasdaq.com/articles/stock-market-news-for-feb-5-2018-2018-02-05
✅ Completed in 63.34 seconds

[2025-03-03 20:10:59] Fetching: https://www.bbc.com/news/business-42942921
✅ Completed in 8.01 seconds

[2025-03-03 20:11:07] Fetching: https://www.cnbc.com/2018/02/05/why-the-stock-market-plunged-today.html
✅ Completed in 18.24 seconds

[2025-03-03 20:11:26] Fet

KeyboardInterrupt: 

In [15]:
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.microsoft import EdgeChromiumDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.EdgeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Selenium WebDriver (Switching to Edge)
service = Service(EdgeChromiumDriverManager().install())  
driver = webdriver.Edge(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from August 2018
start_year = 2018
end_year = max(article["Date"].year for article in all_data)  # Get latest year

# Ensure that we properly filter data to start **AFTER JULY 2018**
all_data = [article for article in all_data if article["Date"].year > 2018 or (article["Date"].year == 2018 and article["Date"].month >= 7)]

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2018 - 2019 (687 articles)

[2025-03-03 21:16:40] Fetching: https://www.investopedia.com/news/merck-stock-seen-rising-12-faster-profit-growth/
✅ Completed in 3.40 seconds

[2025-03-03 21:16:43] Fetching: https://www.fool.com/investing/2018/07/11/what-happened-in-the-stock-market-today.aspx
✅ Completed in 10.18 seconds

[2025-03-03 21:16:54] Fetching: https://www.livemint.com/Money/2MiRpAwHhtzd0nnFRuwNMI/HUL-shares-fall-3-as-analysts-downgrade-stock-after-results.html
✅ Completed in 16.39 seconds

[2025-03-03 21:17:10] Fetching: https://www.thehindubusinessline.com/markets/stock-markets/cyient-drops-to-2-12-week-low-on-weak-results/article24407388.ece
✅ Completed in 9.85 seconds

[2025-03-03 21:17:20] Fetching: https://www.businesstoday.in/markets/stocks/story/sensex-rises-lifetime-high-nifty-holds-above-11000-level-108379-2018-07-18
✅ Completed in 18.15 seconds

[2025-03-03 21:17:38] Fetching: https://money.usnews.com/investing/stock-market-news/articles/2018-07-12

KeyboardInterrupt: 

In [15]:
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.microsoft import EdgeChromiumDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.EdgeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Selenium WebDriver (Switching to Edge)
service = Service(EdgeChromiumDriverManager().install())  
driver = webdriver.Edge(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from August 2018
start_year = 2019
end_year = max(article["Date"].year for article in all_data)  # Get latest year

# Ensure that we properly filter data to start **AFTER JULY 2018**
all_data = [article for article in all_data if article["Date"].year > 2018 or (article["Date"].year == 2018 and article["Date"].month >= 8)]

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2019 - 2020 (914 articles)

[2025-03-03 22:54:02] Fetching: https://www.moneycontrol.com/news/business/exclusive-anil-ambanis-icex-metropolitan-stock-exchange-initiate-merger-talks-3966981.html
✅ Completed in 53.69 seconds

[2025-03-03 22:54:55] Fetching: https://www.washingtonpost.com/business/2019/06/07/heres-why-us-stock-markets-didnt-mind-bit-bad-news/
✅ Completed in 94.30 seconds

[2025-03-03 22:56:30] Fetching: https://www.moneycontrol.com/news/business/markets/mfs-and-fiis-buy-these-16-stocks-consistently-in-last-4-qtrs-what-should-you-do-4333501.html
✅ Completed in 14.26 seconds

[2025-03-03 22:56:44] Fetching: https://www.business-standard.com/article/markets/market-live-sensex-nifty-q1-results-hcl-tech-tata-steel-oil-rupee-119080800112_1.html
✅ Completed in 7.47 seconds

[2025-03-03 22:56:51] Fetching: https://m.economictimes.com/markets/stocks/news/peter-lynchs-5-types-of-stocks-how-to-make-money-in-them/articleshow/70655615.cms
✅ Completed in 55.17 seco


KeyboardInterrupt



In [19]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2020
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2020 - 2021 (913 articles)

[2025-03-04 00:35:01] Fetching: https://www.marketwatch.com/story/stock-market-momentum-is-driving-wall-street-to-uncharted-territory-and-thats-making-even-irrationally-bullish-investors-nervous-2020-01-18
✅ Completed in 100.53 seconds

[2025-03-04 00:36:41] Fetching: https://www.bbc.com/news/business-51796806
✅ Completed in 5.90 seconds

[2025-03-04 00:36:47] Fetching: https://www.livemint.com/market/live-blog/live-blog-sensex-nifty-live-today-20-04-2020-nifty-nse-bse-news-updates-11587350783461.html
✅ Completed in 4.43 seconds

[2025-03-04 00:36:52] Fetching: https://www.marketwatch.com/story/the-stock-market-keeps-rising-while-earnings-keep-falling-what-if-stocks-are-right-2020-05-09
✅ Completed in 77.39 seconds

[2025-03-04 00:38:09] Fetching: https://m.economictimes.com/markets/stocks/news/mysterious-2572-stock-rally-has-traders-scratching-their-heads/articleshow/75675776.cms
❌ Error: HTTPConnectionPool(host='localhost', port=64484)

In [35]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2020
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2020 - 2021 (913 articles)

[2025-03-04 16:02:35] Fetching: https://m.economictimes.com/markets/stocks/news/mysterious-2572-stock-rally-has-traders-scratching-their-heads/articleshow/75675776.cms
✅ Completed in 86.18 seconds

[2025-03-04 16:04:01] Fetching: https://www.moneycontrol.com/news/business/markets/broader-market-underperform-but-these-34-smallcap-stocks-rose-10-30-in-a-week-5304381.html
✅ Completed in 15.15 seconds

[2025-03-04 16:04:16] Fetching: https://indianexpress.com/article/business/market/bse-sensex-nse-nifty-markets-live-updates-stocks-shares-global-cues-6503066/
✅ Completed in 9.90 seconds

[2025-03-04 16:04:26] Fetching: https://www.business-standard.com/article/markets/market-live-sensex-nifty-bse-nse-sgx-nifty-covid-19-tcs-irctc-120071000142_1.html
✅ Completed in 10.25 seconds

[2025-03-04 16:04:36] Fetching: https://www.cnbc.com/2020/07/13/stock-market-futures-open-to-close-news.html
✅ Completed in 12.57 seconds

[2025-03-04 16:04:49] Fetchi

KeyboardInterrupt: 

In [36]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2021
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2021 - 2022 (913 articles)

[2025-03-04 18:21:48] Fetching: https://www.businesstoday.in/markets/market-perspective/story/sensex-hits-50000-index-rose-9000-in-over-34-year-roller-coaster-ride-285127-2021-01-21
✅ Completed in 44.72 seconds

[2025-03-04 18:22:33] Fetching: https://www.india.com/business/share-market-today-news-stock-market-bse-sensex-live-nse-nifty-top-stocks-4577977/
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.india.com/business/share-market-today-news-stock-market-bse-sensex-live-nse-nifty-top-stocks-4577977/
✅ Completed in 107.83 seconds

[2025-03-04 18:24:22] Fetching: https://www.businesstoday.in/opinion/columns/story/how-herd-behaviour-in-stock-market-could-compound-investors-financial-woes-301425-2021-07-15
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://ww


KeyboardInterrupt



In [37]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2021
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2021 - 2022 (913 articles)

[2025-03-04 21:09:08] Fetching: https://www.india.com/business/share-market-today-news-stock-market-bse-sensex-live-nse-nifty-top-stocks-4577977/
✅ Completed in 69.64 seconds

[2025-03-04 21:10:17] Fetching: https://www.businesstoday.in/opinion/columns/story/how-herd-behaviour-in-stock-market-could-compound-investors-financial-woes-301425-2021-07-15
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.businesstoday.in/opinion/columns/story/how-herd-behaviour-in-stock-market-could-compound-investors-financial-woes-301425-2021-07-15
✅ Completed in 62.15 seconds

[2025-03-04 21:11:20] Fetching: https://www.reuters.com/world/india/indian-shares-open-lower-metal-auto-stocks-drop-2021-09-20/
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Erro

KeyboardInterrupt: 

In [38]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2021
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2021 - 2022 (913 articles)

[2025-03-04 21:33:05] Fetching: https://www.businesstoday.in/opinion/columns/story/how-herd-behaviour-in-stock-market-could-compound-investors-financial-woes-301425-2021-07-15
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.businesstoday.in/opinion/columns/story/how-herd-behaviour-in-stock-market-could-compound-investors-financial-woes-301425-2021-07-15
✅ Completed in 65.61 seconds

[2025-03-04 21:34:10] Fetching: https://www.reuters.com/world/india/indian-shares-open-lower-metal-auto-stocks-drop-2021-09-20/
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.reuters.com/world/india/indian-shares-open-lower-metal-auto-stocks-drop-2021-09-20/
✅ Completed in 66.25 seconds

[2025-03-04 21:35:17] Fetching: https://www.reuters.com/world/india/indias-bse-sensex

KeyboardInterrupt: 

In [38]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2022
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2022 - 2023 (910 articles)

[2025-03-04 22:26:51] Fetching: https://www.reuters.com/markets/asia/live-markets-all-it-took-was-few-minutes-market-tanked-2022-01-05/
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.reuters.com/markets/asia/live-markets-all-it-took-was-few-minutes-market-tanked-2022-01-05/
✅ Completed in 66.75 seconds

[2025-03-04 22:27:57] Fetching: https://www.reuters.com/world/india/indian-shares-slip-fifth-day-jsw-steel-drops-2022-01-24/
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Error: No text extracted, retrying...
❌ Timeout: https://www.reuters.com/world/india/indian-shares-slip

In [43]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager  
from datetime import datetime

# Configure WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

def get_full_article_text(article_url, driver):
    """Fetch full text of an article from its URL. Handles retries."""
    start_time = time.time()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Fetching: {article_url}")

    extracted_text = ""
    while extracted_text == "":
        if time.time() - start_time > 60:
            print(f"❌ Timeout: {article_url}")
            break
        try:
            driver.get(article_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            try:
                article_elem = driver.find_element(By.TAG_NAME, "article")
                extracted_text = article_elem.text
            except NoSuchElementException:
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                extracted_text = "\n".join([p.text for p in paragraphs if p.text.strip()])
            if extracted_text.strip() == "":
                raise Exception("No text extracted, retrying...")
        except TimeoutException:
            print(f"❌ Timeout: {article_url}")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(10)
            continue
        break

    elapsed_time = time.time() - start_time
    print(f"✅ Completed in {elapsed_time:.2f} seconds\n")
    return extracted_text

# 🔹 Resume from 2018
start_year = 2023
end_year = max(article["Date"].year for article in all_data)  # Get latest year

for year in range(start_year, end_year + 1, 2):  # Step in 2-year batches
    batch = [article for article in all_data if year <= article["Date"].year < year + 2]

    print(f"\n🔵 Processing Batch: {year} - {year + 1} ({len(batch)} articles)\n")

    for article in batch:
        if "FullText" in article and article["FullText"]:  # Skip already processed articles
            continue
        article["FullText"] = get_full_article_text(article["Link"], driver)

# Quit driver after extraction
driver.quit()



🔵 Processing Batch: 2023 - 2024 (911 articles)

[2025-03-05 00:27:04] Fetching: https://www.wsj.com/livecoverage/stock-market-news-today-03-17-2023
❌ Error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00F20B43+25139]
	(No symbol) [0x00EB13F4]
	(No symbol) [0x00D904E3]
	(No symbol) [0x00D96F01]
	(No symbol) [0x00D9900A]
	(No symbol) [0x00D99087]
	(No symbol) [0x00DD30EE]
	(No symbol) [0x00DFCFCC]
	(No symbol) [0x00DCD9A4]
	(No symbol) [0x00DFD244]
	(No symbol) [0x00E1E778]
	(No symbol) [0x00DFCDC6]
	(No symbol) [0x00DCBDE9]
	(No symbol) [0x00DCD124]
	GetHandleVerifier [0x01224373+3185251]
	GetHandleVerifier [0x0124291A+3309578]
	GetHandleVerifier [0x0123CF42+3286578]
	GetHandleVerifier [0x00FB7AE0+643536]
	(No symbol) [0x00EBA2

In [47]:
from datetime import datetime

# Convert string date to datetime object (assuming date is stored as a string)
target_date = datetime(2024,1,15)

# Filter articles from the target date
articles_on_target_date = [article for article in all_data if article["Date"] == target_date]

# Print results
if articles_on_target_date:
    for article in articles_on_target_date:
        print(f"Title: {article['Title']}")
        print(f"Date: {article['Date'].strftime('%d-%b-%Y')}")
        print(f"Link: {article['Link']}")
        print("-" * 50)
else:
    print("No articles found from February 3, 2015.")


Title: Indian stock market: Here are 6 things that changed for market overnight - Gift Nifty, inflation to Q3 results
Date: 15-Jan-2024
Link: https://www.livemint.com/market/stock-market-news/indian-stock-market-here-are-6-things-that-changed-for-market-overnight-gift-nifty-inflation-to-q3-results-11705279937001.html
--------------------------------------------------


In [48]:
print(f"Total articles scraped: {len(all_data)}")
for article in all_data[:10]:  # Print first 5 articles
    print("\nTitle:", article["Title"])
    print("Date:", article["Date"])
    print("Link:", article["Link"])
    print("FullText:", article["FullText"][:500])  # Print first 500 characters


Total articles scraped: 3015

Title: Japan dethrones China to become world’s second-biggest stock market
Date: 2018-08-03 00:00:00
Link: https://www.hindustantimes.com/business-news/japan-dethrones-china-to-become-world-s-second-biggest-stock-market/story-ogOhBhBTX0Fw3HQEqcX4yN.html
FullText: China just lost its ranking as the world’s number two stock market.
After a Thursday slump, Chinese equities were worth $6.09 trillion, according to data compiled by Bloomberg. That compares with $6.17 trillion in Japan. The US has the world’s largest stock market at just over $31 trillion.
China’s stock market overtook Japan’s in late 2014, then soared to an all-time high of more than $10 trillion in June 2015. Chinese equities and the nation’s currency have taken a beating this year amid a tra

Title: From Rs 20 to Rs 2,000 in 15 years of stock market trading! This blue-chip stock can gain 12% more
Date: 2018-08-02 00:00:00
Link: https://www.financialexpress.com/market/from-rs-20-to-rs-2000-in-1

In [51]:
import pandas as pd
import os

# Load DataFrame directly from memory if already available
if 'all_data' in locals() or 'all_data' in globals():
    df = pd.DataFrame(all_data)


# Ensure data is sorted by Date
df.sort_values(by="Date", inplace=True)

# Get the current working directory
csv_path = os.path.join(os.getcwd(), "googlenews_articles1.csv")

# Save the final dataset
df.to_csv(csv_path, index=False)

# Print the saved file path
print(f"✅ CSV saved at: {csv_path}")
print(f"📂 You can open it from this location.")


✅ CSV saved at: C:\Users\HP\googlenews_articles1.csv
📂 You can open it from this location.


In [53]:
import pandas as pd
import os

# Check if 'all_data' exists before proceeding
if 'all_data' in locals() or 'all_data' in globals():
    df = pd.DataFrame(all_data)
else:
    print("❌ Error: 'all_data' not found!")
    exit()

# Convert "Date" column to proper datetime format and remove time
df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime('%d-%m-%Y')

# Ensure data is sorted by Date
df.sort_values(by="Date", inplace=True)

# Get the current working directory
csv_path = os.path.join(os.getcwd(), "googlenews_articles_new.csv")

# Save the final dataset
df.to_csv(csv_path, index=False)

# Print the saved file path
print(f"✅ CSV saved at: {csv_path}")
print(f"📂 You can open it from this location.")


✅ CSV saved at: C:\Users\HP\googlenews_articles_new.csv
📂 You can open it from this location.


In [54]:
import pandas as pd
import os

# Check if 'all_data' exists before proceeding
if 'all_data' in locals() or 'all_data' in globals():
    df = pd.DataFrame(all_data)
else:
    print("❌ Error: 'all_data' not found!")
    exit()

# Convert "Date" column to datetime and remove time
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# **Sort by Date (Earliest to Latest)**
df.sort_values(by="Date", inplace=True)

# Convert back to "DD-MM-YYYY" format for CSV
df["Date"] = df["Date"].dt.strftime('%d-%m-%Y')

# Get the current working directory
csv_path = os.path.join(os.getcwd(), "googlenews_articles_neww.csv")

# Save the final dataset
df.to_csv(csv_path, index=False)

# Print the saved file path
print(f"✅ CSV saved at: {csv_path}")
print(f"📂 You can open it from this location.")


✅ CSV saved at: C:\Users\HP\googlenews_articles_neww.csv
📂 You can open it from this location.
